# Assignment 4

**Due to**: TBD

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Fact checking, Neural Languange Inference (**NLI**)

# Intro

This assignment is centred on a particular and emerging NLP task, formally known as **fact checking** (or fake checking). As AI techniques become more and more powerful, reaching amazing results, such as image and text generation, it is more than ever necessary to build tools able to distinguish what is real from what is fake.

Here we focus on a small portion of the whole fact checking problem, which aims to determine whether a given statement (fact) conveys a trustworthy information or not. 

More precisely, given a set of evidences and a fact to verify, we would like our model to correctly predict whether the fact is true or fake.

In particular, we will see:

*   Dataset preparation (analysis and pre-processing)
*   Problem formulation: multi-input binary classification
*   Defining an evaluation method
*   Simple sentence embedding
*   Neural building blocks
*   Neural architecture extension

# The FEVER dataset

First of all, we need to choose a dataset. In this assignment we will rely on the [FEVER dataset](https://fever.ai).

The dataset is about facts taken from Wikipedia documents that have to be verified. In particular, facts could face manual modifications in order to define fake information or to give different formulations of the same concept.

The dataset consists of 185,445 claims manually verified against the introductory sections of Wikipedia pages and classified as ```Supported```, ```Refuted``` or ```NotEnoughInfo```. For the first two classes, systems and annotators need to also return the combination of sentences forming the necessary evidence supporting or refuting the claim.

## Dataset structure

Relevant data is divided into two file types. Information concerning the fact to verify, its verdict and associated supporting/opposing statements are stored in **.jsonl** format. In particular, each JSON element is a python dictionary with the following relevant fields:

*    **ID**: ID associated to the fact to verify.

*    **Verifiable**: whether the fact has been verified or not: ```VERIFIABLE``` (or ```NOT VERIFIABLE```).
    
*    **Label**: the final verdict on the fact to verify: ```SUPPORTS```, ```REFUTES``` (or ```NOT ENOUGH INFO```).
    
*    **Claim**: the fact to verify.
    
*    **Evidence**: a nested list of document IDs along with the sentence ID that is associated to the fact to verify. In particular, each list element is a tuple of four elements: the first two are internal annotator IDs that can be safely ignored; the third term is the document ID (called URL) and the last one is the sentence number (ID) in the pointed document to consider.

**Some Examples**

---

**Verifiable**

{"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}

---

**Not Verifiable**

{"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}

---

## Some simplifications and pre-processing

We are only interested in verifiable facts. Thus, we can filter out all non-verifiable claims.

Additionally, the current dataset format does not contain all necessary information for our classification purposes. In particular, we need to download Wikipedia documents and replace reported evidence IDs with the corresponding text.

Don't worry about that! We are providing you the already pre-processed dataset so that you can concentrate on the classification pipeline (pre-processing, model definition, evaluation and training).

You can download the zip file containing all set splits (train, validation and test) of the FEVER dataset by clicking on this [link](https://drive.google.com/file/d/1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1/view?usp=sharing). Alternatively, run the below code cell to automatically download it on this notebook.

**Note**: each dataset split is in .csv format. Feel free to inspect the whole dataset!

In [2]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

# Classification dataset

At this point, you should have a reay-to-go dataset! Note that the dataset format changed as well! In particular, we split the evidence set associated to each claim, in order to build (claim, evidence) pairs. The classification label is propagated as well.

We'll motivate this decision in the next section!

Just for clarity, here's an example of the pre-processed dataset:

---

**Claim**: "Wentworth Miller is yet to make his screenwriting debut."

**Evidence**: "2	He made his screenwriting debut with the 2013 thriller film Stoker .	Stoker	Stoker (film)"

**Label**: Refutes

---

**Note**: The dataset requires some text cleaning as you may notice!


# Problem formulation

As mentioned at the beginning of the assignment, we are going to formulate the fact checking problem as a binary classification task.

In particular, each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

## Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: [batch_size, max_tokens]
*    Evidence: [batch_size, max_tokens]

Moreover, after the embedding layer, we'll have:

*    Claim: [batch_size, max_tokens, embedding_dim]
*    Evidence: [batch_size, max_tokens, embedding_dim]

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: [batch_size, dim]

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

*   Encode token sequences via a RNN and take the last state as the sentence embedding.

*   Encode token sequences via a RNN and average all the output states.

*   Encode token sequences via a simple MLP layer. In particular, if your input is a [batch_size, max_tokens, embedding_dim] tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a [batch_size, embedding_dim] 2D matrix.

*   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## from here

In [1]:
import pandas as pd
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from matplotlib import pyplot as plt

from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000

import seaborn as sns

from gensim.models import KeyedVectors

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [90]:
train_df = pd.read_csv('dataset/train_pairs.csv')
val_df = pd.read_csv('dataset/val_pairs.csv')
test_df = pd.read_csv('dataset/test_pairs.csv')

In [3]:
train_df.head()

,Unnamed: 0,Claim,Evidence,ID,Label
0,0,Chris Hemsworth appeared in A Perfect Getaway.,2\tHemsworth has also appeared in the science ...,3,SUPPORTS
1,1,Roald Dahl is a writer.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,7,SUPPORTS
2,2,Roald Dahl is a governor.,0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ˈ...,8,REFUTES
3,3,Ireland has relatively low-lying mountains.,10\tThe island 's geography comprises relative...,9,SUPPORTS
4,4,Ireland does not have relatively low-lying mou...,10\tThe island 's geography comprises relative...,10,REFUTES


In [4]:
# data cleaning

In [92]:
import re
from functools import reduce
import nltk
from nltk.corpus import stopwords

# Config

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def lower(text):
    return text.lower()

def replace_special_characters(text):
    return REPLACE_BY_SPACE_RE.sub(' ', text)

def filter_out_uncommon_symbols(text):
    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text):
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])

def strip_text(text):
    return text.strip()

def remove_numbers(text):
    return ''.join([i for i in text if not i.isdigit()])

PREPROCESSING_PIPELINE = [
                          lower,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text,
                          remove_numbers
                          ]


def text_prepare(text, filter_methods=None):
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing
print('Pre-processing text...')
# Replace each sentence with its pre-processed version
train_df['Claim'] = train_df['Claim'].apply(lambda txt: text_prepare(txt))
train_df['Evidence'] = train_df['Evidence'].apply(lambda txt: text_prepare(txt))

val_df['Claim'] = val_df['Claim'].apply(lambda txt: text_prepare(txt))
val_df['Evidence'] = val_df['Evidence'].apply(lambda txt: text_prepare(txt))

test_df['Claim'] = val_df['Claim'].apply(lambda txt: text_prepare(txt))
test_df['Evidence'] = val_df['Evidence'].apply(lambda txt: text_prepare(txt))

print("Pre-processing completed!")

Pre-processing text...
Pre-processing completed!


In [6]:
# build vocabulary

In [95]:
def build_voc(df):
    voc_inverted = {}
    current_index = 0
    df_text = np.array(df)
    for i in range(df_text.shape[0]):
        df_text_split = df_text[i].split(' ')
        for word in df_text_split:
            if word not in voc_inverted:
                voc_inverted[word] = current_index
                current_index += 1
    
    voc = {v: k for k, v in voc_inverted.items()}
    
    return voc, voc_inverted, voc_inverted.keys()
# idx_to_word, word_to_idx, word_listing
train_voc_claim = build_voc(train_df['Claim'])
train_voc_evidence = build_voc(train_df['Evidence'])

val_voc_claim = build_voc(val_df['Claim'])
val_voc_evidence = build_voc(val_df['Evidence'])

#test_voc_claim = build_voc(test_df['Claim'])
#test_voc_evidence = build_voc(test_df['Evidence'])

In [10]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(model_type, embedding_dimension=300):
    download_path = ""

    # Find the correct embedding model name
    if model_type.strip().lower() == 'word2vec':
        download_path = "word2vec-google-news-300"

    elif model_type.strip().lower() == 'glove':
        download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)

    else:
        raise AttributeError("Unsupported embedding model type! Available ones: word2vec, glove")

    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

embedding_model_type = "glove"
embedding_dimension = 50

embedding_model = load_embedding_model(embedding_model_type, embedding_dimension)

In [96]:
def check_OOV_terms(embedding_model, word_listing):
    model_vocab_array = np.array(list(embedding_model.vocab.keys()))
    word_listing_array = np.array(list(word_listing))
    return list(np.setdiff1d(word_listing_array, model_vocab_array))
# claim
train_oov_terms_claim = check_OOV_terms(embedding_model, train_voc_claim[2]) # voc_claim[2] represents
                                                                       # the word listing
# evidence
train_oov_terms_evidence = check_OOV_terms(embedding_model, train_voc_evidence[2])


# claim
val_oov_terms_claim = check_OOV_terms(embedding_model, val_voc_claim[2])
# evidence
val_oov_terms_evidence = check_OOV_terms(embedding_model, val_voc_evidence[2])

In [97]:
def build_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, oov_terms):
    words = np.array(list(word_to_idx.keys()))
    embedding_matrix = np.zeros((len(words), embedding_dimension))    
    for idx, w in enumerate(words):
        if w not in oov_terms:
            embedding_matrix[idx, :] = np.array(embedding_model.wv[w])
        else:
           # random vectors
            mu, sigma = 0, 0.1 # mean and standard deviation
            embedding_matrix[idx, :] = np.random.normal(mu, sigma, embedding_dimension)
    return embedding_matrix
train_embedding_matrix_claim = build_embedding_matrix(embedding_model, embedding_dimension, train_voc_claim[1], train_oov_terms_claim)
train_embedding_matrix_evidence = build_embedding_matrix(embedding_model, embedding_dimension, train_voc_evidence[1], train_oov_terms_evidence)

val_embedding_matrix_claim = build_embedding_matrix(embedding_model, embedding_dimension, val_voc_claim[1], val_oov_terms_claim)
val_embedding_matrix_evidence = build_embedding_matrix(embedding_model, embedding_dimension, val_voc_evidence[1], val_oov_terms_evidence)

In [14]:
def create_3D_matrix(train_df, voc_df, max_length, embedding_matrix, embedding_dimension):
    train_df_array = np.array(train_df)
    # zero padding
    three_dim_matrix = np.zeros((train_df_array.shape[0], max_length, embedding_dimension))
    for i in range(train_df_array.shape[0]):
        df_claim_row = train_df_array[i].split(' ')
        for idx, word in enumerate(df_claim_row):
            three_dim_matrix[i, idx, :] = embedding_matrix[voc_df[1][word]]
    return three_dim_matrix

In [15]:
max_length = 150

In [16]:
three_dim_matrix_claim = create_3D_matrix(train_df['Claim'], voc_claim, max_length, embedding_matrix_claim, embedding_dimension)

In [17]:
three_dim_matrix_evidence = create_3D_matrix(train_df['Evidence'], voc_evidence, max_length, embedding_matrix_evidence, embedding_dimension)

In [18]:
three_dim_matrix_claim.shape

(121740, 150, 50)

In [19]:
three_dim_matrix_claim[0, :, :].shape

(150, 50)

In [20]:
three_dim_matrix_evidence.shape

(121740, 150, 50)

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Bidirectional, \
                                    LSTM, GRU, SimpleRNN, Lambda, TimeDistributed, \
                                    Masking, GlobalMaxPool1D, GlobalAveragePooling1D

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
tf.random.set_seed(42)

In [60]:
## test

In [106]:
# for trining data
word_tokenizer = Tokenizer()                      # instantiate tokeniser
word_tokenizer.fit_on_texts(train_df['Claim'])    # fit tokeniser on data
train_df_claim_enc = word_tokenizer.texts_to_sequences(train_df['Claim'])

word_tokenizer = Tokenizer()                      
word_tokenizer.fit_on_texts(train_df['Evidence'])
train_df_evidence_enc = word_tokenizer.texts_to_sequences(train_df['Evidence'])

# for val data
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(val_df['Claim'])
val_df_claim_enc = word_tokenizer.texts_to_sequences(val_df['Claim'])

word_tokenizer = Tokenizer()                      
word_tokenizer.fit_on_texts(val_df['Evidence'])
val_df_evidence_enc = word_tokenizer.texts_to_sequences(val_df['Evidence'])

In [107]:
train_embedding_layer_claim = Embedding(
    train_embedding_matrix_claim.shape[0],    # vocab size 
    train_embedding_matrix_claim.shape[1],    # embedding dimension
    embeddings_initializer=tf.initializers.Constant(train_embedding_matrix_claim),
    mask_zero = True,
    name = 'Embedding_claim',
    trainable = False
)

train_embedd_sentences_claim = Sequential([
  Input((max_length,)),
  train_embedding_layer_claim,
  Bidirectional(LSTM(64, dropout=0.1, return_sequences=True)),
  GlobalAveragePooling1D(),
  Dense(max_length),
])

train_embedding_layer_evidence = Embedding(
    train_embedding_matrix_evidence.shape[0],    # vocab size 
    train_embedding_matrix_evidence.shape[1],    # embedding dimension
    embeddings_initializer=tf.initializers.Constant(train_embedding_matrix_evidence),
    mask_zero = True,
    name = 'Embedding_evidence',
    trainable = False
)

train_embedd_sentences_evidence = Sequential([
  Input((max_length,)),
  train_embedding_layer_evidence,
  Bidirectional(LSTM(64, dropout=0.1, return_sequences=True)),
  GlobalAveragePooling1D(),
  Dense(max_length),
])

In [108]:
val_embedding_layer_claim = Embedding(
    val_embedding_matrix_claim.shape[0],    # vocab size 
    val_embedding_matrix_claim.shape[1],    # embedding dimension
    embeddings_initializer=tf.initializers.Constant(val_embedding_matrix_claim),
    mask_zero = True,
    name = 'Embedding_claim',
    trainable = False
)

val_embedd_sentences_claim = Sequential([
  Input((max_length,)),
  val_embedding_layer_claim,
  Bidirectional(LSTM(64, dropout=0.1, return_sequences=True)),
  GlobalAveragePooling1D(),
  Dense(max_length),
])

val_embedding_layer_evidence = Embedding(
    val_embedding_matrix_evidence.shape[0],    # vocab size 
    val_embedding_matrix_evidence.shape[1],    # embedding dimension
    embeddings_initializer=tf.initializers.Constant(val_embedding_matrix_evidence),
    mask_zero = True,
    name = 'Embedding_evidence',
    trainable = False
)

val_embedd_sentences_evidence = Sequential([
  Input((max_length,)),
  val_embedding_layer_evidence,
  Bidirectional(LSTM(64, dropout=0.1, return_sequences=True)),
  GlobalAveragePooling1D(),
  Dense(max_length),
])

In [ ]:
## embedd all claim matrix

In [109]:
#length_df = len(train_df.index)
length_df = 100 # for testing use just 100
train_claim_2D = np.zeros((length_df, max_length))
for i in range(length_df):
    train_claim_2D[i, :] = train_embedd_sentences_claim(pad_sequences(train_df_claim_enc, padding='post')[i].reshape(1, -1))
    
train_evidence_2D = np.zeros((length_df, max_length))
for i in range(length_df):
    train_evidence_2D[i, :] = train_embedd_sentences_evidence(pad_sequences(train_df_evidence_enc, padding='post')[i].reshape(1, -1))

In [ ]:
#length_df = len(train_df.index)
length_df = 100 # for testing use just 100
val_claim_2D = np.zeros((length_df, max_length))
for i in range(length_df):
    val_claim_2D[i, :] = val_embedd_sentences_claim(pad_sequences(val_df_claim_enc, padding='post')[i].reshape(1, -1))
    
val_evidence_2D = np.zeros((length_df, max_length))
for i in range(length_df):
    val_evidence_2D[i, :] = val_embedd_sentences_evidence(pad_sequences(val_df_evidence_enc, padding='post')[i].reshape(1, -1))

## Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if we the sentence embedding of a single input has shape [batch_size, embedding_dim], then the classification input has shape:

*     **Concatenation**: [batch_size, 2 * embedding_dim]

*     **Sum**: [batch_size, embedding_dim]

*     **Mean**: [batch_size, embedding_dim]

In [ ]:
## concatenation

In [103]:
train_claim_evidence_concat = np.hstack((train_claim_2D, train_evidence_2D))
val_claim_evidence_concat = np.hstack((val_claim_2D, val_evidence_2D))

In [104]:
train_claim_evidence_concat.shape, val_claim_evidence_concat.shape

(100, 300)

In [91]:
## training

In [81]:
train_df["Label"] = train_df["Label"].apply(lambda x: 1 if x=="SUPPORTS" else 0)
val_df["Label"] = val_df["Label"].apply(lambda x: 1 if x=="SUPPORTS" else 0)

In [84]:
y_train = np.array(train_df["Label"])
y_val = np.array(val_df["Label"])

In [85]:
model = Sequential()
model.add(Dense(12, input_dim=train_claim_evidence_concat.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [86]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [89]:
model.fit(train_claim_evidence_concat, y[:100], epochs=200, batch_size=10, validation_data=(val_claim_evidence_concat, y_val))

Epoch 1/200
100/100 [==============================] - 0s 239us/sample - loss: 0.5439 - accuracy: 0.9100
Epoch 2/200
100/100 [==============================] - 0s 121us/sample - loss: 0.4990 - accuracy: 0.9100
Epoch 3/200
100/100 [==============================] - 0s 246us/sample - loss: 0.4467 - accuracy: 0.9100
Epoch 4/200
100/100 [==============================] - 0s 257us/sample - loss: 0.4076 - accuracy: 0.9100
Epoch 5/200
100/100 [==============================] - 0s 233us/sample - loss: 0.3791 - accuracy: 0.9100
Epoch 6/200
100/100 [==============================] - 0s 319us/sample - loss: 0.3526 - accuracy: 0.9100
Epoch 7/200
100/100 [==============================] - 0s 200us/sample - loss: 0.3392 - accuracy: 0.9100
Epoch 8/200
100/100 [==============================] - 0s 200us/sample - loss: 0.3273 - accuracy: 0.9100
Epoch 9/200
100/100 [==============================] - 0s 185us/sample - loss: 0.3186 - accuracy: 0.9100
Epoch 10/200
100/100 [==============================] -

100/100 [==============================] - 0s 200us/sample - loss: 0.1235 - accuracy: 0.9500
Epoch 79/200
100/100 [==============================] - 0s 184us/sample - loss: 0.1207 - accuracy: 0.9500
Epoch 80/200
100/100 [==============================] - 0s 197us/sample - loss: 0.1201 - accuracy: 0.9500
Epoch 81/200
100/100 [==============================] - 0s 192us/sample - loss: 0.1185 - accuracy: 0.9500
Epoch 82/200
100/100 [==============================] - 0s 211us/sample - loss: 0.1171 - accuracy: 0.9500
Epoch 83/200
100/100 [==============================] - 0s 231us/sample - loss: 0.1154 - accuracy: 0.9500
Epoch 84/200
100/100 [==============================] - 0s 181us/sample - loss: 0.1142 - accuracy: 0.9500
Epoch 85/200
100/100 [==============================] - 0s 203us/sample - loss: 0.1124 - accuracy: 0.9500
Epoch 86/200
100/100 [==============================] - 0s 227us/sample - loss: 0.1129 - accuracy: 0.9500
Epoch 87/200
100/100 [==============================] - 0s 

100/100 [==============================] - 0s 198us/sample - loss: 0.0697 - accuracy: 0.9500
Epoch 155/200
100/100 [==============================] - 0s 258us/sample - loss: 0.0682 - accuracy: 0.9600
Epoch 156/200
100/100 [==============================] - 0s 229us/sample - loss: 0.0692 - accuracy: 0.9500
Epoch 157/200
100/100 [==============================] - 0s 202us/sample - loss: 0.0687 - accuracy: 0.9600
Epoch 158/200
100/100 [==============================] - 0s 193us/sample - loss: 0.0690 - accuracy: 0.9600
Epoch 159/200
100/100 [==============================] - 0s 216us/sample - loss: 0.0709 - accuracy: 0.9500
Epoch 160/200
100/100 [==============================] - 0s 203us/sample - loss: 0.0680 - accuracy: 0.9600
Epoch 161/200
100/100 [==============================] - 0s 214us/sample - loss: 0.0680 - accuracy: 0.9600
Epoch 162/200
100/100 [==============================] - 0s 219us/sample - loss: 0.0701 - accuracy: 0.9400
Epoch 163/200
100/100 [============================

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: [batch_size, 2 * embedding_dim + 1]

*     **Sum**: [batch_size, embedding_dim + 1]

*     **Mean**: [batch_size, embedding_dim + 1]



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

**Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

**Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

# Tips and Extras

## Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

Don't forget to try neural baselines first!

## Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

Don't forget that your feedback is very important! Your suggestions help us improving course material.

# FAQ

---

**Q: Can I do something text pre-processing?**

**A:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Q: I'm struggling with the implementation. Can you help me?**

**A:** Yes sure! Write us an email about your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Q: Can I try other encoding strategies or neural architectures?**

**A:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---